# Prototype - Intelligent Home Care System

Quick prototype to test both sub-systems before full implementation:
- Fall Detection (LSTM vs GRU)
- ECG Anomaly Detection (LSTM Autoencoder vs GRU Autoencoder)

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

np.random.seed(42)
tf.random.set_seed(42)

SISFALL_PATH = Path(r'C:\Users\MYP\Desktop\Parthiban\Uni\CS_AI\2026_Term_4\ICT304\Assignment\Assignment 1\datasets\SisFall_dataset')
ECG_PATH = Path(r'C:\Users\MYP\Desktop\Parthiban\Uni\CS_AI\2026_Term_4\ICT304\Assignment\Assignment 1\datasets\Heartbeat')

print(f'TF version: {tf.__version__}')
print(f'SisFall found: {SISFALL_PATH.exists()}')
print(f'ECG found: {ECG_PATH.exists()}')

TF version: 2.20.0
SisFall found: True
ECG found: True


## Part 1 - Fall Detection (LSTM vs GRU)

In [14]:
# helper to parse sisfall files (comma separated, semicolon at end of each line)
def load_sisfall_file(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    data = []
    for line in lines:
        line = line.strip().rstrip(';')
        if line:
            vals = [int(x.strip()) for x in line.split(',')]
            data.append(vals)
    return np.array(data)

In [15]:
# loading a small subset for the prototype (3 subjects, 15 files each for falls/ADLs)
X_fall, X_adl = [], []

for subj in ['SA01', 'SA02', 'SA03']:
    subj_path = SISFALL_PATH / subj
    all_files = list(subj_path.glob('*.txt'))

    fall_files = [f for f in all_files if f.name.startswith('F')][:15]
    adl_files = [f for f in all_files if f.name.startswith('D')][:15]

    for file in fall_files + adl_files:
        try:
            data = load_sisfall_file(file)
            mid = len(data) // 2
            window = data[mid-100:mid+100] if len(data) > 200 else data[:200]
            if len(window) == 200:
                if file.name.startswith('F'):
                    X_fall.append(window)
                else:
                    X_adl.append(window)
        except:
            pass

print(f'falls: {len(X_fall)}, adl: {len(X_adl)}')

falls: 45, adl: 45


In [16]:
# balance + normalize + split
n = min(len(X_fall), len(X_adl))
X = np.array(X_fall[:n] + X_adl[:n])
y = np.array([1]*n + [0]*n)  # 1=fall, 0=adl

idx = np.random.permutation(len(X))
X, y = X[idx], y[idx]

scaler = StandardScaler()
X_norm = scaler.fit_transform(X.reshape(-1, 9)).reshape(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)
print(f'train: {X_train.shape}, test: {X_test.shape}')

train: (72, 200, 9), test: (18, 200, 9)


In [17]:
# LSTM
lstm = Sequential([
    LSTM(32, input_shape=(200, 9)),
    Dense(1, activation='sigmoid')
])
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2, verbose=1)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.7368 - loss: 0.6283 - val_accuracy: 0.6667 - val_loss: 0.6673
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7368 - loss: 0.5850 - val_accuracy: 0.6667 - val_loss: 0.6648
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7544 - loss: 0.5502 - val_accuracy: 0.6667 - val_loss: 0.6639
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7719 - loss: 0.5214 - val_accuracy: 0.6667 - val_loss: 0.6647
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7895 - loss: 0.4975 - val_accuracy: 0.6667 - val_loss: 0.6668
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7895 - loss: 0.4774 - val_accuracy: 0.6667 - val_loss: 0.6697
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8070 - loss: 0.4599 - val_accuracy: 0.6667 - val_loss: 0.6727
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7895 - loss: 0.4440 - val_accuracy: 0.6667 - val_loss: 0.6751


In [18]:
# GRU
gru = Sequential([
    GRU(32, input_shape=(200, 9)),
    Dense(1, activation='sigmoid')
])
gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2, verbose=1)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.5965 - loss: 0.6805 - val_accuracy: 0.6000 - val_loss: 0.6780
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7368 - loss: 0.6448 - val_accuracy: 0.5333 - val_loss: 0.6759
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7719 - loss: 0.6145 - val_accuracy: 0.5333 - val_loss: 0.6740
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7368 - loss: 0.5879 - val_accuracy: 0.5333 - val_loss: 0.6722
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7368 - loss: 0.5648 - val_accuracy: 0.5333 - val_loss: 0.6706
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7544 - loss: 0.5446 - val_accuracy: 0.5333 - val_loss: 0.6690
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7895 - loss: 0.5269 - val_accuracy: 0.5333 - val_loss: 0.6676
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7895 - loss: 0.5115 - val_accuracy: 0.5333 - val_loss: 0.6662

In [19]:
# results
lstm_pred = (lstm.predict(X_test, verbose=0) > 0.5).astype(int).flatten()
gru_pred = (gru.predict(X_test, verbose=0) > 0.5).astype(int).flatten()

print('Fall Detection Results')
print(f'  LSTM -> acc: {accuracy_score(y_test, lstm_pred):.4f}, f1: {f1_score(y_test, lstm_pred):.4f}')
print(f'  GRU  -> acc: {accuracy_score(y_test, gru_pred):.4f}, f1: {f1_score(y_test, gru_pred):.4f}')

Fall Detection Results
  LSTM -> acc: 0.7222, f1: 0.6154
  GRU  -> acc: 0.7222, f1: 0.6154


## Part 2 - ECG Anomaly Detection (LSTM AE vs GRU AE)

In [20]:
# load ecg data - randomly sample 5000 (the csv is sorted by class)
train_df = pd.read_csv(ECG_PATH / 'mitbih_train.csv', header=None)
train_df = train_df.sample(n=5000, random_state=42).reset_index(drop=True)

X_ecg = train_df.iloc[:, :-1].values
y_ecg = train_df.iloc[:, -1].values
y_binary = (y_ecg != 0).astype(int)  # 0=normal, 1=anomaly

print(f'ecg shape: {X_ecg.shape}')
print(f'normal: {sum(y_binary==0)}, anomaly: {sum(y_binary==1)}')

ecg shape: (5000, 187)
normal: 4180, anomaly: 820


In [21]:
# autoencoders are trained only on normal heartbeats
# the idea: if it only learns "normal", anomalies will have higher reconstruction error
X_normal = X_ecg[y_binary == 0].reshape(-1, 187, 1)

X_train_ae, X_val_ae = train_test_split(X_normal, test_size=0.2, random_state=42)
print(f'training on {len(X_train_ae)} normal heartbeats')

training on 3344 normal heartbeats


In [22]:
# LSTM autoencoder
# using tanh instead of relu to avoid the NaN issue with relu on autoencoders
lstm_ae = Sequential([
    LSTM(32, activation='tanh', input_shape=(187, 1), return_sequences=False),
    RepeatVector(187),
    LSTM(32, activation='tanh', return_sequences=True),
    TimeDistributed(Dense(1))
])
lstm_ae.compile(optimizer='adam', loss='mse')
lstm_ae.fit(X_train_ae, X_train_ae, epochs=10, batch_size=32, validation_data=(X_val_ae, X_val_ae), verbose=1)

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step - loss: 0.0458 - val_loss: 0.0432
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - loss: 0.0401 - val_loss: 0.0380
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - loss: 0.0321 - val_loss: 0.0280
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - loss: 0.0274 - val_loss: 0.0257
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - loss: 0.0248 - val_loss: 0.0250
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - loss: 0.0243 - val_loss: 0.0237
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 94ms/step - loss: 0.0235 - val_loss: 0.0232
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - loss: 0.0220 - val_loss: 0.0189
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 87ms/step - loss: 0.0184 - val_loss: 0.0169
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 87ms/step - loss: 0.0167 - val_loss: 0.0157


In [23]:
# GRU autoencoder
gru_ae = Sequential([
    GRU(32, activation='tanh', input_shape=(187, 1), return_sequences=False),
    RepeatVector(187),
    GRU(32, activation='tanh', return_sequences=True),
    TimeDistributed(Dense(1))
])
gru_ae.compile(optimizer='adam', loss='mse')
gru_ae.fit(X_train_ae, X_train_ae, epochs=10, batch_size=32, validation_data=(X_val_ae, X_val_ae), verbose=1)

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 131ms/step - loss: 0.0496 - val_loss: 0.0484
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 112ms/step - loss: 0.0451 - val_loss: 0.0435
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 97ms/step - loss: 0.0395 - val_loss: 0.0337
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - loss: 0.0320 - val_loss: 0.0292
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 101ms/step - loss: 0.0236 - val_loss: 0.0214
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - loss: 0.0197 - val_loss: 0.0192
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 115ms/step - loss: 0.0177 - val_loss: 0.0176
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 100ms/step - loss: 0.0172 - val_loss: 0.0171
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step - loss: 0.0169 - val_loss: 0.0169
Epoch 10/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - loss: 0.0165 - val_loss: 0.0166


In [24]:
# anomaly detection via reconstruction error
X_all = X_ecg.reshape(-1, 187, 1)

lstm_recon = lstm_ae.predict(X_all, verbose=0)
gru_recon = gru_ae.predict(X_all, verbose=0)

lstm_error = np.mean((X_all - lstm_recon)**2, axis=(1, 2))
gru_error = np.mean((X_all - gru_recon)**2, axis=(1, 2))

# threshold = 95th percentile of reconstruction error on normal samples
lstm_thresh = np.percentile(lstm_error[y_binary == 0], 95)
gru_thresh = np.percentile(gru_error[y_binary == 0], 95)

lstm_pred_ae = (lstm_error > lstm_thresh).astype(int)
gru_pred_ae = (gru_error > gru_thresh).astype(int)

print('ECG Anomaly Detection Results')
print(f'  LSTM AE -> acc: {accuracy_score(y_binary, lstm_pred_ae):.4f}, f1: {f1_score(y_binary, lstm_pred_ae):.4f}')
print(f'  GRU AE  -> acc: {accuracy_score(y_binary, gru_pred_ae):.4f}, f1: {f1_score(y_binary, gru_pred_ae):.4f}')

ECG Anomaly Detection Results
  LSTM AE -> acc: 0.8348, f1: 0.3295
  GRU AE  -> acc: 0.8142, f1: 0.1771


## Summary

In [25]:
print('Prototype Results')
print()
print('Fall Detection:')
print(f'  LSTM  {accuracy_score(y_test, lstm_pred)*100:.1f}% acc')
print(f'  GRU   {accuracy_score(y_test, gru_pred)*100:.1f}% acc')
print()
print('ECG Anomaly Detection:')
print(f'  LSTM AE  {accuracy_score(y_binary, lstm_pred_ae)*100:.1f}% acc')
print(f'  GRU AE   {accuracy_score(y_binary, gru_pred_ae)*100:.1f}% acc')
print()
print('this is just a small prototype - full version will use the complete dataset with tuning')

Prototype Results

Fall Detection:
  LSTM  72.2% acc
  GRU   72.2% acc

ECG Anomaly Detection:
  LSTM AE  83.5% acc
  GRU AE   81.4% acc

this is just a small prototype - full version will use the complete dataset with tuning
